In [1]:
import sys 
sys.path.append("..") 

import numpy as np
import pandas as pd
from SWMM import SWMM_ENV
from PPO import Buffer
from PPO import PPO as PPO
import tensorflow as tf
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import os

In [2]:
np.load('time.npy',allow_pickle=True)

array([ 332.8678901,  662.0102699, 6740.517343 ])

In [2]:
pd.read_csv(os.path.dirname(os.getcwd())+'/SWMM/action_table.csv').values[:,1:].shape

(16, 6)

In [6]:
env_params={
        'orf':os.path.dirname(os.getcwd())+'/SWMM/chaohu',
        'orf_save':'chaohu_RTC',
        'parm':os.path.dirname(os.getcwd())+'/states_yaml/chaohu',
        'advance_seconds':300,
        'kf':1,
        'kc':1,
        'reward_type':'3'
    }
env=SWMM_ENV.SWMM_ENV(env_params)
agent_params={
    'state_dim':len(env.config['states']),
    'action_dim':int(2**len(env.config['action_assets'])),
    'actornet_layer':[30,30,30],
    'criticnet_layer':[30,30,30],
    
    'bound_low':0,
    'bound_high':1,
    
    'clip_ratio':0.01,
    'target_kl':0.03,
    'lam':0.01,
    
    'policy_learning_rate':0.005,
    'value_learning_rate':0.005,
    'train_policy_iterations':20,
    'train_value_iterations':20,
    
    'num_rain':50,
    
    'training_step':1000,
    'gamma':0.3,
    'epsilon':1,
    'ep_min':1e-50,
    'ep_decay':0.1,

    'action_table':pd.read_csv(os.path.dirname(os.getcwd())+'/SWMM/action_table.csv').values[:,1:],
}

In [3]:
optdata = np.load(os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'\\pretraindata\\OPT_results\\opt_results_rain'+str(0)+'.npy',
                        allow_pickle=True).tolist()
o=optdata['state'][1:-2]

In [8]:
raindata = np.load(os.path.dirname(os.getcwd())+'/rainfall/training_raindata.npy').tolist()
observation, episode_return, episode_length = env.reset(raindata[0],0,True,os.path.dirname(os.getcwd())), 0, 0

In [11]:
observation = np.array(observation).reshape(1, -1)
tem_model.critic(np.array(observation))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.5958906], dtype=float32)>

In [17]:
res = []
tem_model = PPO.PPO(agent_params)
tem_model.load_model('./model/')
tem_model.params['epsilon']=tem_model.params['epsilon']
for rainid in range(50):
    episode_return, episode_length = 0, 0
    optdata = np.load(os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'\\pretraindata\\OPT_results\\opt_results_rain'+str(rainid)+'.npy',
                        allow_pickle=True).tolist()
    o=optdata['state'][1:-2]
    a=optdata['action'][1:-1]
    r=optdata['rewards'][1:-2]
    o_=optdata['state'][2:-1]
    last_value=0
    vt, lo = [], []
    for observation in o:
        value_t = tem_model.critic(np.array(observation).reshape(1, -1)).numpy()[0]
        logits, action = PPO.sample_action(np.array(observation).reshape(1, -1),tem_model,True)
        logprobability_t = PPO.logprobabilities(logits, action, tem_model.params['action_dim']).numpy()[0]
        vt.append(value_t)
        lo.append(logprobability_t)

    episode_return += np.sum(r)
    episode_length += len(r)
    res.append([o,a,r,vt,lo,last_value,episode_return,episode_length])

In [19]:
res[0][4]

[-0.01837119,
 -0.021702426,
 -0.0034974378,
 -0.0032056395,
 -0.0031552552,
 -0.00437256,
 -0.0060556983,
 -5.6327815,
 -0.019726405,
 -0.26338056,
 -7.0358706,
 -0.006602141,
 -0.0063394406,
 -0.012600571,
 -0.013280506,
 -0.51601565,
 -0.017824585,
 -0.02092535,
 -0.022957142,
 -9.108603,
 -0.023903985,
 -0.02692147,
 -0.032211192,
 -0.031648885,
 -0.039366238,
 -0.023903985,
 -0.032960285,
 -0.05086006,
 -0.058366355,
 -0.048965853,
 -0.048965853,
 -0.031648885,
 -0.025936361,
 -0.025936361,
 -0.025232432,
 -0.025232665,
 -0.025232665,
 -0.077420704,
 -0.13729374,
 -0.07970145,
 -0.40908134,
 -0.010059493,
 -0.18460067,
 -0.031424798,
 -0.007274805,
 -0.006338019,
 -0.22126018,
 -0.04413114,
 -0.10706564,
 -0.054227374,
 -0.05272694,
 -0.051785275,
 -0.019901244,
 -0.010296559,
 -0.0074306536,
 -0.021576319,
 -0.36276004,
 -0.051710453,
 -0.4284015,
 -0.37766033,
 -0.12961148,
 -0.016129367,
 -0.008164128,
 -0.0073886467,
 -10.02993,
 -0.026603112,
 -0.042337492,
 -0.121897824,
 -0